# HISTORY

#### Context
* Solar flares are intense bursts of radiation which can disrupt the power grids of a continent, shut down the GPS system or irradiate people exposed in space.
* Developing systems for predicting solar flares would allow us to precisely aim our observation instruments at upcoming events, and eventually enable countermeasures against such worst-case scenarios.

#### Acknowledgements
* This dataset was created by Roman Bolzern and Michael Aerni from the Institute for Data Science, FHNW, Switzerland. We owe our thanks to the SDO satellite mission, and to JSOC Stanford for providing the raw data.

#### Inspiration
* The prediction of solar flares proves to be a challenging problem, some even compare it to weather forecasting. And regarding Machine Learning, we find this dataset to be particularly challenging because of the complexity of a single sample (up to 40 images), the relatively small size of samples (8'000 for training), and the fact that it is a regression problem. Yet Kagglers have proven time and time again that predictions can be made on the most complex of data. By providing this dataset, we hope to encourage Kaggle machine learners to push the envelope of solar flare predictions.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/2/28/Sunspot_Numbers.png/400px-Sunspot_Numbers.png)

![](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5e/Solar_Cycle_Prediction.gif/400px-Solar_Cycle_Prediction.gif)

# PACKAGES AND LIBRARIES

In [ ]:
#GENERAL
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import random
import time
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
from scipy.io import loadmat
import nibabel as nib
import csv
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing import image
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
from scipy.ndimage.filters import convolve
from skimage import data, io, filters
import skimage
from skimage.morphology import convex_hull_image, erosion
from IPython import display
from scipy.ndimage import gaussian_filter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.patches as patches
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D,Reshape, Conv2DTranspose,\
LeakyReLU, GaussianNoise, GlobalMaxPooling2D, ReLU, Input, Concatenate
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
from keras.datasets import mnist
import keras
from keras.models import Model
#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

# OVERVIEW

#### MAIN PATH

In [ ]:
Main_Path = Path("../input/sdobenchmark/SDOBenchmark_example/training")

#### CSV PROCESS

In [ ]:
Main_Train_CSV = list(Main_Path.glob(r"**/*.csv"))

In [ ]:
print(len(Main_Train_CSV))

In [ ]:
Train_CSV = Main_Train_CSV[0]

In [ ]:
print(Train_CSV)

In [ ]:
Reading_CSV = pd.read_csv(Train_CSV)

In [ ]:
print(Reading_CSV.columns)

In [ ]:
print(len(Reading_CSV))

In [ ]:
print(Reading_CSV.head(-1))

In [ ]:
print(Reading_CSV.isnull().sum())

In [ ]:
print(Reading_CSV["peak_flux"][0])

In [ ]:
print(int(Reading_CSV["peak_flux"][0]))

In [ ]:
print(int(Reading_CSV["peak_flux"][414]))

In [ ]:
print(Reading_CSV["start"][414])

In [ ]:
print(Reading_CSV["start"][414][0:10])

#### DATETIME TRANSFORMATION

In [ ]:
print(type(Reading_CSV["start"][414][0:10]))

In [ ]:
Testing_Date_Transformation = Reading_CSV["start"][414][0:10].replace("-","")

In [ ]:
print(Testing_Date_Transformation)

In [ ]:
Testing_Date_Array = np.array(int(Testing_Date_Transformation))

In [ ]:
print(type(Testing_Date_Array))

# DATA PROCESS

#### ENGINEERING

In [ ]:
JPG_Path = list(Main_Path.glob(r"**/*.jpg"))

In [ ]:
Sorted_JPG_Path = sorted(JPG_Path)

In [ ]:
Reading_CSV["New_ID"] = Sorted_JPG_Path[40:457]

In [ ]:
print(Reading_CSV.head(-1))

In [ ]:
print(Reading_CSV["New_ID"][0])
print("---"*10)
print(Reading_CSV["id"][0])

#### NEW DATA

In [ ]:
New_Start = []
New_End = []
New_Path = []

for x_start,x_end,x_path in zip(Reading_CSV.start.values,Reading_CSV.end.values,Reading_CSV.New_ID.values):
    
    x_start = x_start[0:10]
    x_start = x_start.replace("-","")
    x_start = np.array(x_start,dtype="float32")
    
    x_end = x_end[0:10]
    x_end = x_end.replace("-","")
    x_end = np.array(x_end,dtype="float32")
    
    New_Start.append(x_start)
    New_End.append(x_end)
    New_Path.append(x_path)
    

In [ ]:
print("LEN START: ",len(New_Start))
print("LEN END: ",len(New_End))
print("LEN PATH: ",len(New_Path))

In [ ]:
print(New_Start[0])
print("--"*10)
print(New_End[0])
print("--"*10)
print(New_Path[0])

In [ ]:
print(type(New_Start[0]))
print("--"*10)
print(type(New_End[0]))
print("--"*10)
print(type(New_Path[0]))

#### TO DATAFRAME

In [ ]:
Start_Series = pd.Series(New_Start,name="START").astype(np.float32)
End_Series = pd.Series(New_End,name="END").astype(np.float32)
Path_Series = pd.Series(New_Path,name="PATH").astype(str)

In [ ]:
Main_Data = pd.concat([Path_Series,Start_Series,End_Series],axis=1)

In [ ]:
print(Main_Data.head(-1))

# VISION

#### SIMPLE

In [ ]:
Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][33]),cv2.COLOR_BGR2RGB)

figure = plt.figure(figsize=(10,10))

plt.xlabel(Example_IMG.shape)
plt.ylabel(Example_IMG.size)
plt.imshow(Example_IMG)

#### MULTI

In [ ]:
figure,axis = plt.subplots(5,5,figsize=(10,10))

for indexing,operations in enumerate(axis.flat):
    
    Reading_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][indexing]),cv2.COLOR_BGR2RGB)
    
    operations.set_xlabel(Reading_IMG.shape)
    operations.set_ylabel(Reading_IMG.size)
    operations.imshow(Reading_IMG)

plt.tight_layout()
plt.show()

#### 2D

In [ ]:
Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][13]),cv2.COLOR_BGR2RGB)

figure = plt.figure(figsize=(10,10))

plt.xlabel(Example_IMG[:,:,0].shape)
plt.ylabel(Example_IMG[:,:,0].size)
plt.imshow(Example_IMG[:,:,0])

#### THRESHOLD

In [ ]:
Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][13]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)

figure = plt.figure(figsize=(10,10))

plt.xlabel(Threshold_IMG.shape)
plt.ylabel(Threshold_IMG.size)
plt.imshow(Threshold_IMG)

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][13]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)

axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Threshold_IMG)
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][23]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)

axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Threshold_IMG)
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][2]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)

axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Threshold_IMG)
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][404]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)

axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Threshold_IMG)
axis[1].axis("off")

#### HESSIAN SPECTRUM

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(10,10))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][404]),cv2.COLOR_BGR2GRAY)
Hessian_IMG = hessian_matrix(Example_IMG,sigma=0.20,order="rc")
Max_IMG,Min_IMG = hessian_matrix_eigvals(Hessian_IMG)

axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Max_IMG)
axis[1].axis("off")
axis[2].imshow(Min_IMG)
axis[2].axis("off")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(10,10))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][44]),cv2.COLOR_BGR2GRAY)
Hessian_IMG = hessian_matrix(Example_IMG,sigma=0.20,order="rc")
Max_IMG,Min_IMG = hessian_matrix_eigvals(Hessian_IMG)

axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Max_IMG)
axis[1].axis("off")
axis[2].imshow(Min_IMG)
axis[2].axis("off")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(10,10))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][55]),cv2.COLOR_BGR2GRAY)
Hessian_IMG = hessian_matrix(Example_IMG,sigma=0.20,order="rc")
Max_IMG,Min_IMG = hessian_matrix_eigvals(Hessian_IMG)

axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Max_IMG)
axis[1].axis("off")
axis[2].imshow(Min_IMG)
axis[2].axis("off")

#### CANNY-THRESHOLD CONCAT

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(10,10))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][55]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)
Canny_IMG = cv2.Canny(Threshold_IMG,10,100)


axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Threshold_IMG)
axis[1].axis("off")
axis[2].imshow(Canny_IMG)
axis[2].axis("off")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(10,10))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][333]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)
Canny_IMG = cv2.Canny(Threshold_IMG,10,100)


axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Threshold_IMG)
axis[1].axis("off")
axis[2].imshow(Canny_IMG)
axis[2].axis("off")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(15,15))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][333]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)
Canny_IMG = cv2.Canny(Threshold_IMG,10,100)
Concat_IMG = cv2.addWeighted(Example_IMG[:,:,0],0.8,Canny_IMG,0.4,0.5)

axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Canny_IMG)
axis[1].axis("off")
axis[2].imshow(Concat_IMG)
axis[2].axis("off")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(15,15))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][13]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)
Canny_IMG = cv2.Canny(Threshold_IMG,10,100)
Concat_IMG = cv2.addWeighted(Example_IMG[:,:,0],0.8,Canny_IMG,0.4,0.5)

axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Canny_IMG)
axis[1].axis("off")
axis[2].imshow(Concat_IMG)
axis[2].axis("off")

#### CONTOURS

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(15,15))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][13]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)
Canny_IMG = cv2.Canny(Threshold_IMG,10,100)
Concat_IMG = cv2.addWeighted(Example_IMG[:,:,0],0.8,Canny_IMG,0.4,0.5)
contours,_ = cv2.findContours(Canny_IMG,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    x,y,w,h = cv2.boundingRect(contour)
    Rect_IMG = cv2.rectangle(Concat_IMG,(x,y),(x+w,y+h),(255,0,0),2)
    
axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Concat_IMG,cmap="hot")
axis[1].axis("off")
axis[2].imshow(Threshold_IMG)
axis[2].axis("off")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(15,15))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][113]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)
Canny_IMG = cv2.Canny(Threshold_IMG,10,100)
Concat_IMG = cv2.addWeighted(Example_IMG[:,:,0],0.8,Canny_IMG,0.4,0.5)
contours,_ = cv2.findContours(Canny_IMG,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    x,y,w,h = cv2.boundingRect(contour)
    Rect_IMG = cv2.rectangle(Concat_IMG,(x,y),(x+w,y+h),(255,0,0),2)
    
axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Concat_IMG,cmap="hot")
axis[1].axis("off")
axis[2].imshow(Threshold_IMG)
axis[2].axis("off")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(15,15))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][188]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)
Canny_IMG = cv2.Canny(Threshold_IMG,10,100)
Concat_IMG = cv2.addWeighted(Example_IMG[:,:,0],0.8,Canny_IMG,0.4,0.5)
contours,_ = cv2.findContours(Canny_IMG,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    x,y,w,h = cv2.boundingRect(contour)
    Rect_IMG = cv2.rectangle(Concat_IMG,(x,y),(x+w,y+h),(255,0,0),2)
    
axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Concat_IMG,cmap="hot")
axis[1].axis("off")
axis[2].imshow(Threshold_IMG)
axis[2].axis("off")

#### LAYER CONCAT

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(15,15))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][13]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)
Canny_IMG = cv2.Canny(Threshold_IMG,10,100)

Copy_Main = Example_IMG.copy()
Copy_Main[Canny_IMG == 1] = [0,255,0]
Copy_Main[Canny_IMG == 2] = [0,0,255]
Copy_Main_Two = Example_IMG.copy()

Concat_IMG = cv2.addWeighted(Copy_Main,0.5,Copy_Main_Two,0.7,0,Copy_Main_Two)

axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Canny_IMG)
axis[1].axis("off")
axis[2].imshow(Concat_IMG)
axis[2].axis("off")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(15,15))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][13]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)
Canny_IMG = cv2.Canny(Threshold_IMG,10,100)

Copy_Main = Example_IMG.copy()
Copy_Main[Canny_IMG == 255] = [255,0,0]

axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Canny_IMG)
axis[1].axis("off")
axis[2].imshow(Copy_Main)
axis[2].axis("off")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(15,15))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][123]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)
Canny_IMG = cv2.Canny(Threshold_IMG,10,100)

Copy_Main = Example_IMG.copy()
Copy_Main[Canny_IMG == 255] = [255,0,0]

axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Canny_IMG)
axis[1].axis("off")
axis[2].imshow(Copy_Main)
axis[2].axis("off")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(15,15))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][223]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)
Canny_IMG = cv2.Canny(Threshold_IMG,10,100)

Copy_Main = Example_IMG.copy()
Copy_Main[Threshold_IMG[:,:,0] == 255] = [255,0,0]

axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Threshold_IMG)
axis[1].axis("off")
axis[2].imshow(Copy_Main)
axis[2].axis("off")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(15,15))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][1]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)
Canny_IMG = cv2.Canny(Threshold_IMG,10,100)

Copy_Main = Example_IMG.copy()
Copy_Main[Threshold_IMG[:,:,0] == 255] = [255,0,0]

axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Threshold_IMG)
axis[1].axis("off")
axis[2].imshow(Copy_Main)
axis[2].axis("off")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(15,15))

Example_IMG = cv2.cvtColor(cv2.imread(Main_Data["PATH"][17]),cv2.COLOR_BGR2RGB)
_,Threshold_IMG = cv2.threshold(Example_IMG,200,255,cv2.THRESH_BINARY)
Canny_IMG = cv2.Canny(Threshold_IMG,10,100)

Copy_Main = Example_IMG.copy()
Copy_Main[Threshold_IMG[:,:,0] == 255] = [255,0,0]

axis[0].imshow(Example_IMG)
axis[0].axis("off")
axis[1].imshow(Threshold_IMG)
axis[1].axis("off")
axis[2].imshow(Copy_Main)
axis[2].axis("off")

# DATA TRANSFORMATION

#### FOR PREDICTION

In [ ]:
X_Start = []
X_End = []
X_Image = []

for img,start_i,end_i in zip(Main_Data.PATH.values,Main_Data.START.values,Main_Data.END.values):
    Picking_IMG = cv2.cvtColor(cv2.imread(img),cv2.COLOR_BGR2RGB)
    Picking_IMG = cv2.resize(Picking_IMG,(180,180))
    Picking_IMG = Picking_IMG / 255.
    
    X_Image.append(Picking_IMG)
    X_Start.append(start_i)
    X_End.append(end_i)

In [ ]:
print(np.shape(np.array(X_Image)))
print(np.shape(np.array(X_Start)))
print(np.shape(np.array(X_End)))

In [ ]:
Train_JPG = np.array(X_Image,dtype="float32")
Train_START = np.array(X_Start,dtype="float32")
Train_END = np.array(X_End,dtype="float32")

In [ ]:
print(Train_JPG.shape)
print(Train_START.shape)
print(Train_END.shape)

In [ ]:
Scalar_Function = MinMaxScaler()

Train_START_R = Scalar_Function.fit_transform(Train_START.reshape(-1,1))
Train_END_R = Scalar_Function.fit_transform(Train_END.reshape(-1,1))

#### FOR AUTO-ENCODER

In [ ]:
X_Mask = []
X_New_IMG = []

for img_i in Main_Data.PATH.values:
    Picking_IMG = cv2.cvtColor(cv2.imread(img_i),cv2.COLOR_BGR2RGB)
    _,Threshold_IMG = cv2.threshold(Picking_IMG,200,255,cv2.THRESH_BINARY)
    Canny_IMG = cv2.Canny(Threshold_IMG,10,100)

    Copy_Main = Picking_IMG.copy()
    Copy_Main[Canny_IMG == 255] = [255,0,0]
    
    Copy_Main = cv2.resize(Copy_Main,(180,180))
    Copy_Main = Copy_Main / 255.
    
    Picking_IMG = cv2.resize(Picking_IMG,(180,180))
    Picking_IMG = Picking_IMG / 255.
    
    X_Mask.append(Copy_Main)
    X_New_IMG.append(Picking_IMG)

In [ ]:
X_Mask = np.array(X_Mask,dtype="float32")
X_New_IMG = np.array(X_New_IMG,dtype="float32")

In [ ]:
print(X_New_IMG.shape)
print(X_Mask.shape)

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))

axis[0].imshow(X_New_IMG[0])
axis[0].axis("off")
axis[1].imshow(X_Mask[0])
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))

axis[0].imshow(X_New_IMG[100])
axis[0].axis("off")
axis[1].imshow(X_Mask[100])
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))

axis[0].imshow(X_New_IMG[200])
axis[0].axis("off")
axis[1].imshow(X_Mask[200])
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))

axis[0].imshow(X_New_IMG[231])
axis[0].axis("off")
axis[1].imshow(X_Mask[231])
axis[1].axis("off")

# AUTO ENCODER MODEL

#### CALLBACKS & PARAMETERS

In [ ]:
Early_Stopper = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=3,mode="min")
Checkpoint_Model = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      filepath="./modelcheck")
Reduce_Model = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy",
                                                   factor=0.1,
                                                   patience=5)

In [ ]:
compile_loss = "binary_crossentropy"
compile_optimizer = Adam(lr=0.0000001)
output_class = 1

#### STRUCTURE

In [ ]:
E_AE = Sequential()
#
E_AE.add(Conv2D(32,(5,5),kernel_initializer = 'he_normal',use_bias = True, padding="valid"))
E_AE.add(BatchNormalization())
E_AE.add(ReLU())
#
E_AE.add(Conv2D(64,(5,5),kernel_initializer = 'he_normal',use_bias = True, padding="valid"))
E_AE.add(BatchNormalization())
E_AE.add(ReLU())
#
E_AE.add(Conv2D(128,(5,5),kernel_initializer = 'he_normal',use_bias = True, padding="valid"))
E_AE.add(BatchNormalization())
E_AE.add(ReLU())
#
E_AE.add(Conv2D(256,(5,5),kernel_initializer = 'he_normal',use_bias = True, padding="valid"))
E_AE.add(BatchNormalization())
E_AE.add(ReLU())




D_AE = Sequential()
#
D_AE.add(Conv2DTranspose(128,(5,5), padding="valid"))
D_AE.add(ReLU())
#
D_AE.add(Conv2DTranspose(64,(5,5), padding="valid"))
D_AE.add(ReLU())
#
D_AE.add(Conv2DTranspose(32,(5,5), padding="valid"))
D_AE.add(ReLU())
#
D_AE.add(Conv2DTranspose(output_class,(5,5), padding="valid"))
D_AE.add(ReLU())

In [ ]:
Auto_Encoder = Sequential([E_AE,D_AE])
Auto_Encoder.compile(loss=compile_loss,optimizer=compile_optimizer,metrics=["mse"])

#### TRAINING

In [ ]:
Model_AutoEncoder = Auto_Encoder.fit(X_New_IMG,X_Mask,epochs=55,callbacks=[Checkpoint_Model])

#### PREDICTION

In [ ]:
plt.style.use("dark_background")

In [ ]:
Prediction_MASK_Seen = Auto_Encoder.predict(X_New_IMG[:20])

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))
pre_img_number = 1

Original_Img = X_New_IMG[pre_img_number]
Predict_Mask = Prediction_MASK_Seen[pre_img_number]

axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(Predict_Mask)
axis[1].set_xlabel(Predict_Mask.shape)
axis[1].set_ylabel(Predict_Mask.size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))
pre_img_number = 2

Original_Img = X_New_IMG[pre_img_number]
Predict_Mask = Prediction_MASK_Seen[pre_img_number]

axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(Predict_Mask)
axis[1].set_xlabel(Predict_Mask.shape)
axis[1].set_ylabel(Predict_Mask.size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))
pre_img_number = 3

Original_Img = X_New_IMG[pre_img_number]
Predict_Mask = Prediction_MASK_Seen[pre_img_number]

axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(Predict_Mask)
axis[1].set_xlabel(Predict_Mask.shape)
axis[1].set_ylabel(Predict_Mask.size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))
pre_img_number = 4

Original_Img = X_New_IMG[pre_img_number]
Predict_Mask = Prediction_MASK_Seen[pre_img_number]

axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(Predict_Mask)
axis[1].set_xlabel(Predict_Mask.shape)
axis[1].set_ylabel(Predict_Mask.size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))
pre_img_number = 5

Original_Img = X_New_IMG[pre_img_number]
Predict_Mask = Prediction_MASK_Seen[pre_img_number]

axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(Predict_Mask)
axis[1].set_xlabel(Predict_Mask.shape)
axis[1].set_ylabel(Predict_Mask.size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))
pre_img_number = 6

Original_Img = X_New_IMG[pre_img_number]
Predict_Mask = Prediction_MASK_Seen[pre_img_number]

axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(Predict_Mask)
axis[1].set_xlabel(Predict_Mask.shape)
axis[1].set_ylabel(Predict_Mask.size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))
pre_img_number = 7

Original_Img = X_New_IMG[pre_img_number]
Predict_Mask = Prediction_MASK_Seen[pre_img_number]

axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(Predict_Mask,cmap="hot")
axis[1].set_xlabel(Predict_Mask.shape)
axis[1].set_ylabel(Predict_Mask.size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))
pre_img_number = 8

Original_Img = X_New_IMG[pre_img_number]
Predict_Mask = Prediction_MASK_Seen[pre_img_number]

axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(Predict_Mask,cmap="hot")
axis[1].set_xlabel(Predict_Mask.shape)
axis[1].set_ylabel(Predict_Mask.size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))
pre_img_number = 9

Original_Img = X_New_IMG[pre_img_number]
Predict_Mask = Prediction_MASK_Seen[pre_img_number]

axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(Predict_Mask,cmap="hot")
axis[1].set_xlabel(Predict_Mask.shape)
axis[1].set_ylabel(Predict_Mask.size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))
pre_img_number = 10

Original_Img = X_New_IMG[pre_img_number]
Predict_Mask = Prediction_MASK_Seen[pre_img_number]

axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(Predict_Mask,cmap="jet")
axis[1].set_xlabel(Predict_Mask.shape)
axis[1].set_ylabel(Predict_Mask.size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))
pre_img_number = 11

Original_Img = X_New_IMG[pre_img_number]
Predict_Mask = Prediction_MASK_Seen[pre_img_number]

axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(Predict_Mask,cmap="jet")
axis[1].set_xlabel(Predict_Mask.shape)
axis[1].set_ylabel(Predict_Mask.size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(15,15))
pre_img_number = 12

Original_Img = X_New_IMG[pre_img_number]
Predict_Mask = Prediction_MASK_Seen[pre_img_number]

axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(Predict_Mask,cmap="jet")
axis[1].set_xlabel(Predict_Mask.shape)
axis[1].set_ylabel(Predict_Mask.size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

#### TEST SET PREDICTION

In [ ]:
Reading_IMG = cv2.cvtColor(cv2.imread("../input/sdobenchmark/SDOBenchmark_example/test/11476/2012_05_14_16_18_57_0/2012-05-14T041857__335.jpg"),cv2.COLOR_BGR2RGB)
Reading_IMG = cv2.resize(Reading_IMG,(180,180))

In [ ]:
Reading_IMG_Shape = Reading_IMG.reshape(1,Reading_IMG.shape[0],Reading_IMG.shape[1],Reading_IMG.shape[2])

In [ ]:
figure = plt.figure(figsize=(10,10))

plt.imshow(Reading_IMG)

In [ ]:
MASK_Seen = Auto_Encoder.predict(Reading_IMG_Shape)

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(14,14))

axis[0].imshow(Reading_IMG)
axis[0].set_xlabel(Reading_IMG.shape)
axis[0].set_ylabel(Reading_IMG.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(MASK_Seen[0])
axis[1].set_xlabel(MASK_Seen[0].shape)
axis[1].set_ylabel(MASK_Seen[0].size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

#### RANDOM TEST SET PREDICTION

In [ ]:
Reading_IMG = cv2.cvtColor(cv2.imread("../input/sdobenchmark/SDOBenchmark_example/test/11476/2012_05_14_16_18_57_0/2012-05-14T144857__magnetogram.jpg"),cv2.COLOR_BGR2RGB)
Reading_IMG = cv2.resize(Reading_IMG,(180,180))
Reading_IMG_Shape = Reading_IMG.reshape(1,Reading_IMG.shape[0],Reading_IMG.shape[1],Reading_IMG.shape[2])
MASK_Seen = Auto_Encoder.predict(Reading_IMG_Shape)



figure,axis = plt.subplots(1,2,figsize=(14,14))

axis[0].imshow(Reading_IMG)
axis[0].set_xlabel(Reading_IMG.shape)
axis[0].set_ylabel(Reading_IMG.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(MASK_Seen[0])
axis[1].set_xlabel(MASK_Seen[0].shape)
axis[1].set_ylabel(MASK_Seen[0].size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
Reading_IMG = cv2.cvtColor(cv2.imread("../input/sdobenchmark/SDOBenchmark_example/test/11476/2012_05_14_16_18_57_0/2012-05-14T160857__193.jpg"),cv2.COLOR_BGR2RGB)
Reading_IMG = cv2.resize(Reading_IMG,(180,180))
Reading_IMG_Shape = Reading_IMG.reshape(1,Reading_IMG.shape[0],Reading_IMG.shape[1],Reading_IMG.shape[2])
MASK_Seen = Auto_Encoder.predict(Reading_IMG_Shape)



figure,axis = plt.subplots(1,2,figsize=(14,14))

axis[0].imshow(Reading_IMG)
axis[0].set_xlabel(Reading_IMG.shape)
axis[0].set_ylabel(Reading_IMG.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(MASK_Seen[0],cmap="hot")
axis[1].set_xlabel(MASK_Seen[0].shape)
axis[1].set_ylabel(MASK_Seen[0].size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
Reading_IMG = cv2.cvtColor(cv2.imread("../input/sdobenchmark/SDOBenchmark_example/test/11476/2012_05_14_16_18_57_0/2012-05-14T111857__94.jpg"),cv2.COLOR_BGR2RGB)
Reading_IMG = cv2.resize(Reading_IMG,(180,180))
Reading_IMG_Shape = Reading_IMG.reshape(1,Reading_IMG.shape[0],Reading_IMG.shape[1],Reading_IMG.shape[2])
MASK_Seen = Auto_Encoder.predict(Reading_IMG_Shape)



figure,axis = plt.subplots(1,2,figsize=(14,14))

axis[0].imshow(Reading_IMG)
axis[0].set_xlabel(Reading_IMG.shape)
axis[0].set_ylabel(Reading_IMG.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(MASK_Seen[0],cmap="hot")
axis[1].set_xlabel(MASK_Seen[0].shape)
axis[1].set_ylabel(MASK_Seen[0].size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
Reading_IMG = cv2.cvtColor(cv2.imread("../input/sdobenchmark/SDOBenchmark_example/test/12033/2014_04_10_12_00_00_0/2014-04-10T000000__continuum.jpg"),cv2.COLOR_BGR2RGB)
Reading_IMG = cv2.resize(Reading_IMG,(180,180))
Reading_IMG_Shape = Reading_IMG.reshape(1,Reading_IMG.shape[0],Reading_IMG.shape[1],Reading_IMG.shape[2])
MASK_Seen = Auto_Encoder.predict(Reading_IMG_Shape)



figure,axis = plt.subplots(1,2,figsize=(14,14))

axis[0].imshow(Reading_IMG)
axis[0].set_xlabel(Reading_IMG.shape)
axis[0].set_ylabel(Reading_IMG.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(MASK_Seen[0],cmap="gist_yarg")
axis[1].set_xlabel(MASK_Seen[0].shape)
axis[1].set_ylabel(MASK_Seen[0].size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
Reading_IMG = cv2.cvtColor(cv2.imread("../input/sdobenchmark/SDOBenchmark_example/test/12033/2014_04_10_12_00_00_0/2014-04-10T103000__magnetogram.jpg"),cv2.COLOR_BGR2RGB)
Reading_IMG = cv2.resize(Reading_IMG,(180,180))
Reading_IMG_Shape = Reading_IMG.reshape(1,Reading_IMG.shape[0],Reading_IMG.shape[1],Reading_IMG.shape[2])
MASK_Seen = Auto_Encoder.predict(Reading_IMG_Shape)



figure,axis = plt.subplots(1,2,figsize=(14,14))

axis[0].imshow(Reading_IMG)
axis[0].set_xlabel(Reading_IMG.shape)
axis[0].set_ylabel(Reading_IMG.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(MASK_Seen[0],cmap="cubehelix")
axis[1].set_xlabel(MASK_Seen[0].shape)
axis[1].set_ylabel(MASK_Seen[0].size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
Reading_IMG = cv2.cvtColor(cv2.imread("../input/sdobenchmark/SDOBenchmark_example/test/12473/2015_12_31_06_38_01_0/2015-12-30T183801__335.jpg"),cv2.COLOR_BGR2RGB)
Reading_IMG = cv2.resize(Reading_IMG,(180,180))
Reading_IMG_Shape = Reading_IMG.reshape(1,Reading_IMG.shape[0],Reading_IMG.shape[1],Reading_IMG.shape[2])
MASK_Seen = Auto_Encoder.predict(Reading_IMG_Shape)



figure,axis = plt.subplots(1,2,figsize=(14,14))

axis[0].imshow(Reading_IMG)
axis[0].set_xlabel(Reading_IMG.shape)
axis[0].set_ylabel(Reading_IMG.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(MASK_Seen[0],cmap="cubehelix")
axis[1].set_xlabel(MASK_Seen[0].shape)
axis[1].set_ylabel(MASK_Seen[0].size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

# AUTO ENCODER MODEL II

#### STRUCTURE

In [ ]:
encoder = Sequential()
encoder.add(Flatten(input_shape=[180,180,3]))
encoder.add(Dense(300,activation="relu"))
encoder.add(Dense(200,activation="relu"))
encoder.add(Dense(100,activation="relu"))
encoder.add(Dense(50,activation="relu"))
encoder.add(Dense(25,activation="relu"))

decoder = Sequential()
decoder.add(Dense(50,input_shape=[25],activation="relu"))
decoder.add(Dense(100,activation="relu"))
decoder.add(Dense(200,activation="relu"))
decoder.add(Dense(300,activation="relu"))
decoder.add(Dense(180*180*3,activation="sigmoid"))
decoder.add(Reshape([180,180,3]))

In [ ]:
AutoEncoder_II = Sequential([encoder,decoder])

In [ ]:
AutoEncoder_II.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=["mse"])

#### TRAINING

In [ ]:
AutoEncoder_II.fit(X_New_IMG,X_New_IMG,epochs=40)

#### PREDICTION

In [ ]:
PRE_Img = AutoEncoder_II.predict(X_New_IMG[:10])

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(14,14))
pre_img_number = 1

Original_Img = X_New_IMG[pre_img_number]
Predict_Reduce = PRE_Img[pre_img_number]

axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(Predict_Reduce)
axis[1].set_xlabel(Predict_Reduce.shape)
axis[1].set_ylabel(Predict_Reduce.size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(14,14))
pre_img_number = 2

Original_Img = X_New_IMG[pre_img_number]
Predict_Reduce = PRE_Img[pre_img_number]

axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("IMAGE")
axis[0].axis("off")
axis[1].imshow(Predict_Reduce)
axis[1].set_xlabel(Predict_Reduce.shape)
axis[1].set_ylabel(Predict_Reduce.size)
axis[1].set_title("PREDICTION")
axis[1].axis("off")